
# Lab 9: Build a Log Aggregator

In this lab, you will create your own log generator, build a command-line utility that scans log files, summarizes their contents, and provides insight into system behavior. Data structures to track log message levels such as `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.

This lab reinforces:
- File I/O
- Pattern recognition (regex)
- Dictionaries and counters
- Functions and modularity
- CLI arguments, logging



## Part 1: Create Log files (20%)
Using the the following example log format below create a **python file** that will log errors In a structured tree format 

You will find examples in the folder called Logs that you can use to build your program.

Remember set of logs should have a varied levels of log entries (`INFO`, `WARNING`, `ERROR`, `CRITICAL`) and tailored message types for different service components.
You must create 5 structured logs here are some examples:

    sqldb
    ui
    frontend.js
    backend.js
    frontend.flask
    backend.flask

You may use chat GPT to create sample outputs NOT THE LOGS. IE:

    System failure
    Database corruption
    Disk failure detected
    Database corruption


In [29]:
# Paste your python file here
import logging
import os

# Step 1: Make sure Logs folder exists
os.makedirs("Logs", exist_ok=True)

# Step 2: Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    filename='Logs/smart_home_logs.log',
    filemode='w',
    force=True
)

# Step 3: Define loggers (parent and children)
main_logger = logging.getLogger("home_system")
security_logger = logging.getLogger("home_system.security")
heating_logger = logging.getLogger("home_system.heating")
lighting_logger = logging.getLogger("home_system.lighting")
network_logger = logging.getLogger("home_system.network")

# Step 4: Log events with different levels
main_logger.info("Smart home system initialized")
security_logger.warning("Motion detected at the front door")
heating_logger.error("Temperature sensor not responding")
lighting_logger.info("Living room lights turned off automatically")
network_logger.critical("Wi-Fi connection lost — attempting reconnection")



### Example Log Format

You will work with logs that follow this simplified structure:

```
2025-04-11 23:20:36,913 | my_app | INFO | Request completed
2025-04-11 23:20:36,914 | my_app.utils | ERROR | Unhandled exception
2025-04-11 23:20:36,914 | my_app.utils.db | CRITICAL | Disk failure detected
```


## Part 2: Logging the Log File (40%)
    New File
### Part 2a: Read the Log File (see lab 7) (10%)


Write a function to read the contents of a log file into a list of lines. Handle file errors gracefully.

### Part 2b: Parse Log Lines (see code below if you get stuck) (10%)

Use a regular expression to extract:
- Timestamp
- Log name
- Log level
- Message

### Part 2c: Count Log Levels (20%)

Create a function to count how many times each log level appears. Store the results in a dictionary. Then output it as a Json File
You may pick your own format but here is an example. 
```python
{
    "INFO": 
    {
        "Request completed": 42, 
        "Heartbeat OK": 7
    }

    "WARNING":
    {
        ...
    }
}

```


In [ ]:
# Paste your python file here don't for get to upload it with your submission

In [14]:
from collections import defaultdict
import json
import re

def read_log_file(filepath):
    try:
        with open(filepath, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"Error reading file: {e}")
    return []

def parse_log_line(line):
    pattern = r'^(.*?)\s+\|\s+(.*?)\s+\|\s+(.*?)\s+\|\s+(.*)$'
    match = re.match(pattern, line.strip())
    if match:
        return {
            "timestamp": match.group(1),
            "log_name": match.group(2),
            "level": match.group(3),
            "message": match.group(4)
        }
    return None

def count_log_levels(filepath, output_json="log_summary.json"):
    lines = read_log_file(filepath)
    level_counts = defaultdict(int)

    for line in lines:
        parsed = parse_log_line(line)
        if parsed:
            level = parsed["level"]
            level_counts[level] += 1

    with open(output_json, 'w') as f:
        json.dump(level_counts, f, indent=4)

    print(f"Log summary written to '{output_json}'")
    return level_counts

log_file_path = "Logs/smart_home_logs.log"
summary = count_log_levels(log_file_path)
print(summary)

Log summary written to 'log_summary.json'
defaultdict(<class 'int'>, {'INFO': 2, 'WARNING': 1, 'ERROR': 1, 'CRITICAL': 1})



## Step 3: Generate Summary Report (40%)
    New File
### Step 3a (20%):
 Develop a function that continuously monitors your JSON file(s) and will print a real-time summary of log activity. It should keep count of the messages grouped by log level (INFO, WARNING, ERROR, CRITICAL) and display only the critical messages. (I.e. If new data comes in the summary will change and a new critical message will be printed)
 - note: do not reprocess the entire file on each update.  

### Step 3a: Use a Matplotlib (Lecture 10) (20%)
Develop a function that continuously monitors your JSON file(s) and will graph in real-time a bar or pie plot of each of the errors.  (a graph for each log level). 
- The graph should show the distribution of log messages by level  (INFO, WARNING, ERROR, CRITICAL)  


### Critical notes:
- Your code mus use Daemon Threads (Lecture 14)
- 3a and 3b do not need to run at the same time. 


In [ ]:
import json
import threading
import time
import os

def monitor_log_summary(json_file_path):
    last_seen = {}

    def watch_file():
        print("Starting log monitor ...")
        while True:
            if os.path.exists(json_file_path):
                with open(json_file_path, "r") as f:
                    try:
                        data = json.load(f)
                    except json.JSONDecodeError:
                        time.sleep(1)
                        continue

                # Compare with last seen and print updates
                for level, count in data.items():
                    if level == "CRITICAL":
                        last_count = last_seen.get(level, 0)
                        if count > last_count:
                            print(f"New CRITICAL log detected! Count: {count}")
                    last_seen[level] = count

            time.sleep(2)  # Polling every 2 seconds

    t = threading.Thread(target=watch_file, daemon=True)
    t.start()


In [19]:
import matplotlib.pyplot as plt

def live_plot_log_summary(json_file_path):
    """Continuously update a bar chart of log levels from a JSON file."""
    plt.ion()  # interactive mode on

    def plot_loop():
        while True:
            if os.path.exists(json_file_path):
                with open(json_file_path, "r") as f:
                    try:
                        data = json.load(f)
                    except json.JSONDecodeError:
                        time.sleep(1)
                        continue

                levels = list(data.keys())
                counts = list(data.values())

                plt.clf()
                plt.bar(levels, counts)
                plt.title("Log Level Summary")
                plt.xlabel("Log Level")
                plt.ylabel("Count")
                plt.pause(2)  # update every 2 seconds

            time.sleep(2)

    t = threading.Thread(target=plot_loop, daemon=True)
    t.start()


In [21]:
# Path to your log summary JSON
json_path = "Logs/log_summary.json"

# Run real-time summary monitoring
monitor_log_summary(json_path)

# OR: Run live graph (run one at a time as per instructions)
# live_plot_log_summary(json_path)

# Keep main thread alive
while True:
    time.sleep(10)


Starting log monitor (daemon)...


KeyboardInterrupt: 